<a href="https://colab.research.google.com/github/laresamdeola/Machine-Learning/blob/main/Network_Security_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import math

In [2]:
data = pd.read_csv('UNSW-NB15_1.csv')

<ipython-input-2-adde79575085>:1: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('UNSW-NB15_1.csv')


Preprocessing

In [3]:
data.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,NaN,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 49 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   srcip             700000 non-null  object 
 1   sport             700000 non-null  object 
 2   dstip             700000 non-null  object 
 3   dsport            700000 non-null  object 
 4   proto             700000 non-null  object 
 5   state             700000 non-null  object 
 6   dur               700000 non-null  float64
 7   sbytes            700000 non-null  int64  
 8   dbytes            700000 non-null  int64  
 9   sttl              700000 non-null  int64  
 10  dttl              700000 non-null  int64  
 11  sloss             700000 non-null  int64  
 12  dloss             700000 non-null  int64  
 13  service           700000 non-null  object 
 14  Sload             700000 non-null  float64
 15  Dload             700000 non-null  float64
 16  Spkts             70

In [5]:
data.loc[:, 'srcip']

0         59.166.0.0
1         59.166.0.6
2         59.166.0.5
3         59.166.0.3
4         59.166.0.0
             ...    
699995    59.166.0.8
699996    59.166.0.0
699997    59.166.0.0
699998    59.166.0.6
699999    59.166.0.0
Name: srcip, Length: 700000, dtype: object

In [6]:
data['attack_cat'].fillna('normal', inplace=True)

In [7]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [8]:
data['service'].isnull().sum()

0

In [9]:
data['service']

0            -
1          dns
2          dns
3          dns
4            -
          ... 
699995       -
699996    http
699997       -
699998       -
699999       -
Name: service, Length: 700000, dtype: object

In [10]:
data['service'].replace('-', 'others', inplace=True)

In [11]:
data['state']

0         CON
1         CON
2         CON
3         CON
4         CON
         ... 
699995    FIN
699996    FIN
699997    FIN
699998    CON
699999    FIN
Name: state, Length: 700000, dtype: object

In [12]:
data['service']

0         others
1            dns
2            dns
3            dns
4         others
           ...  
699995    others
699996      http
699997    others
699998    others
699999    others
Name: service, Length: 700000, dtype: object

In [13]:
data['dsport'].value_counts()

dsport
53       107483
80        61279
5190      33429
53        29813
6881      28222
          ...  
13388         1
51505         1
47835         1
14703         1
59241         1
Name: count, Length: 87819, dtype: int64

In [14]:
data['dsport'].replace('0xc0a8', 53, inplace=True)
# data['service'].replace('-', 'others', inplace=True)

In [15]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [16]:
data['dsport'].unique()

array(['1024', '53', '111', ..., 29112, 30186, 59241], dtype=object)

In [17]:
dsport_only = {
    'dsport': data['dsport'].unique()
}

dsport_only_frame = pd.DataFrame(dsport_only)

In [18]:
dsport_only_frame

,dsport
0,1024
1,53
2,111
3,0
4,41049
...,...
87813,20416
87814,29289
87815,29112
87816,30186


In [19]:
# dsport_csv = dsport_only_frame.to_csv('dsport_only.csv')

In [20]:
sport_only = {
    'dsport': data['sport'].unique()
}

sport_only_frame = pd.DataFrame(sport_only)

In [21]:
# sport_csv = sport_only_frame.to_csv('sport_only.csv')

In [22]:
cat_columns = ['srcip', 'dstip', 'sport', 'dsport', 'proto', 'state']

for i in cat_columns:
  if i == 'srcip':
    data[i].replace('-', '59.166.0.2', inplace=True)
  if i == 'dstip':
    data[i].replace('-', '149.171.126.4', inplace=True)
  if i == 'sport':
    # if isinstance(data[i], str):
    data[i].replace('-', 0, inplace=True)
    data[i].replace('0x000c', 0, inplace=True)
    data[i].replace('0x000b', 0, inplace=True)
    data[i].replace('0x20205321', 0, inplace=True)
  if i == 'dsport':
    # if isinstance(i, str):
    data[i].replace('-', 53, inplace=True)
    data[i].replace('0x000c', 0, inplace=True)
    data[i].replace('0x000b', 0, inplace=True)
    data[i].replace('0x20205321', 0, inplace=True)
  if i == 'proto':
    data[i].replace('-', 'tcp', inplace=True)
  if i == 'state':
    data[i].replace('-', 'FIN', inplace=True)

In [23]:
data['is_sm_ips_ports'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 700000 entries, 0 to 699999
Series name: is_sm_ips_ports
Non-Null Count   Dtype
--------------   -----
700000 non-null  int64
dtypes: int64(1)
memory usage: 5.3 MB


In [24]:
data['is_sm_ips_ports']

0         0
1         0
2         0
3         0
4         0
         ..
699995    0
699996    0
699997    0
699998    0
699999    0
Name: is_sm_ips_ports, Length: 700000, dtype: int64

In [25]:
data['ct_ftp_cmd']

0         0
1         0
2         0
3         0
4         0
         ..
699995    0
699996    0
699997    0
699998    0
699999    0
Name: ct_ftp_cmd, Length: 700000, dtype: int64

In [26]:
data['state'].replace('-', 'others', inplace=True)

In [27]:
data['state'].replace('-', 'others', inplace=True)

In [28]:
data['srcip'].value_counts()

srcip
59.166.0.2         67209
59.166.0.0         67127
59.166.0.5         67091
59.166.0.4         66722
59.166.0.1         66587
59.166.0.3         66145
59.166.0.6         64689
59.166.0.8         64640
59.166.0.9         64187
59.166.0.7         63725
175.45.176.1       14325
149.171.126.18      6010
175.45.176.3        5128
175.45.176.0        4782
175.45.176.2        3236
10.40.85.1          1680
10.40.182.1         1670
10.40.85.30          888
10.40.170.2          874
10.40.182.3          874
149.171.126.1        251
149.171.126.5        249
149.171.126.6        241
149.171.126.2        232
149.171.126.3        231
149.171.126.4        225
149.171.126.9        217
149.171.126.8        217
149.171.126.7        207
149.171.126.0        192
192.168.241.243      108
149.171.126.11        16
149.171.126.15         8
149.171.126.16         6
149.171.126.19         3
149.171.126.10         3
149.171.126.12         2
149.171.126.13         1
149.171.126.17         1
127.0.0.1          

In [29]:
data['dstip'].value_counts()

dstip
149.171.126.4      67331
149.171.126.1      66973
149.171.126.0      66925
149.171.126.5      66723
149.171.126.3      66697
149.171.126.2      66674
149.171.126.9      65187
149.171.126.7      64057
149.171.126.6      63951
149.171.126.8      63604
149.171.126.18      8213
175.45.176.1        5965
149.171.126.12      4815
149.171.126.13      2113
149.171.126.17      2109
149.171.126.10      1975
149.171.126.14      1955
224.0.0.5           1912
149.171.126.11      1796
149.171.126.15      1631
10.40.182.3         1582
149.171.126.19      1449
149.171.126.16      1400
10.40.85.1           888
10.40.170.2          874
10.40.85.30          718
59.166.0.1           273
59.166.0.2           253
59.166.0.0           241
59.166.0.9           233
59.166.0.4           218
59.166.0.8           215
59.166.0.5           214
59.166.0.6           211
59.166.0.7           203
59.166.0.3           201
192.168.241.243      108
175.45.176.0          70
224.0.0.1             14
175.45.176.2       

In [30]:
data['sport'].value_counts()

sport
0        6065
47439    4689
1043     4033
0        3611
47439    2243
         ... 
63893       1
22815       1
34459       1
3496        1
34143       1
Name: count, Length: 100269, dtype: int64

In [31]:
data['dsport'].value_counts()

dsport
53       107544
80        61279
5190      33429
53        29813
6881      28222
          ...  
53118         1
6756          1
14493         1
34609         1
59241         1
Name: count, Length: 87816, dtype: int64

In [32]:
data['proto'].value_counts()

proto
tcp     494737
udp     195478
arp       4062
ospf      2116
unas      1442
         ...  
xtp         14
3pc         14
udt          4
esp          2
rtp          1
Name: count, Length: 135, dtype: int64

In [33]:
data['state'].value_counts()

state
FIN    487911
CON    187504
INT     21799
REQ      2429
CLO       111
URH       108
RST        74
ECO        26
ACC        22
PAR         4
TXD         2
URN         2
no          2
MAS         2
TST         2
ECR         2
Name: count, dtype: int64

In [34]:
data['service'].value_counts()

service
others      430656
dns         121169
http         55858
ftp-data     37305
smtp         23588
ftp          16531
ssh          14636
pop3           206
ssl             20
snmp            14
radius           7
dhcp             7
irc              3
Name: count, dtype: int64

In [35]:
hello = '0x000c'

# type(hello) == str

isinstance(hello, str)

True

In [36]:
data['state'].empty

False

In [37]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [38]:
data['ct_dst_src_ltm']

0         2
1         1
2         1
3         1
4         2
         ..
699995    4
699996    2
699997    2
699998    3
699999    2
Name: ct_dst_src_ltm, Length: 700000, dtype: int64

In [39]:
data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,normal,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,normal,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,normal,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,normal,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,59.166.0.8,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,normal,0
699996,59.166.0.0,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,normal,0
699997,59.166.0.0,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,normal,0
699998,59.166.0.6,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,normal,0


In [40]:
data.loc[:, 'Label'].value_counts()

Label
0    677785
1     22215
Name: count, dtype: int64

In [41]:
data.drop(['attack_cat'], axis=1, inplace=True)

In [42]:
data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,0,2,4,2,3,1,1,2,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,0,12,8,1,2,2,1,1,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,0,6,9,1,1,1,1,1,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,0,7,9,1,1,1,1,1,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,0,2,4,2,3,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,59.166.0.8,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,0,8,20,7,5,1,1,4,0
699996,59.166.0.0,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,0,1,1,2,7,2,2,2,0
699997,59.166.0.0,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,0,13,13,6,7,2,1,2,0
699998,59.166.0.6,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,0,10,13,6,5,1,1,3,0


In [43]:
len(data.columns)

48

In [44]:
len(data.columns)

48

In [45]:
data['Label']

0         0
1         0
2         0
3         0
4         0
         ..
699995    0
699996    0
699997    0
699998    0
699999    0
Name: Label, Length: 700000, dtype: int64

In [46]:
len(data.columns)

48

In [47]:
X = data.iloc[:, 0:48].values
y = data.iloc[:, -1].values

In [48]:
X.shape

(700000, 48)

In [49]:
y.shape

(700000,)

In [50]:
X[:, 4:5]

array([['udp'],
       ['udp'],
       ['udp'],
       ...,
       ['tcp'],
       ['udp'],
       ['tcp']], dtype=object)

In [51]:
X[:, 5:6]

array([['CON'],
       ['CON'],
       ['CON'],
       ...,
       ['FIN'],
       ['CON'],
       ['FIN']], dtype=object)

In [52]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

# Encode the "proto" column
X[:, 4:5] = np.array(encoder.fit_transform(X[:, 4:5]))
X[:, 5:6] = np.array(encoder.fit_transform(X[:, 5:6]))
X[:, 13:14] = np.array(encoder.fit_transform(X[:, 13:14]))

In [53]:
len(X[:, 4:5])

700000

In [54]:
X[:, 13:14]

array([[6.0],
       [1.0],
       [1.0],
       ...,
       [6.0],
       [6.0],
       [6.0]], dtype=object)

In [55]:
X[:, 47]

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [56]:
X.shape

(700000, 48)

In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [58]:
len(X[:,0])

700000

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [60]:
X_train.shape

(560000, 130)

In [61]:
X.shape

(700000, 130)

In [62]:
# X_train_scaled = X_train
# X_test_scaled = X_test

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled[:, :-6] = scaler.fit_transform(X_train_scaled[:, :-6])
# X_test_scaled[:, :-6] = scaler.transform(X_test_scaled[:, :-6])

In [63]:
# np.isnan(X)

In [64]:
# len(null_count)

In [65]:
# for i in y_train:
#   if np.isnan(i):
#     i = 0.0

In [66]:
# null_count = []

# for i in y_train:
#   if np.isnan(i):
#     null_count.append(i)

In [67]:
# len(null_count)

In [68]:
from sklearn.naive_bayes import GaussianNB

nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

GaussianNB()

In [69]:
y_pred_nb = nb_classifier.predict(X_test)

In [70]:
len(data.columns)

48